## Build a Financial News and Stock Analysis Agent with Amazon Bedrock Agent
In this notebook, we will build a financial news and stock analysis agent using Amazon Bedrock. The agent will be able to analyze financial news articles and provide insights on stock performance. We will use the following components:
- [**Amazon Bedrock Agents**](https://aws.amazon.com/bedrock/agents/): A fully managed service that makes it easy to build and deploy GenAI powered AI agents.
- [**AWS Cloud Development Kit (CDK)**](https://aws.amazon.com/cdk/): A software development framework for defining cloud infrastructure in code and provisioning it through AWS CloudFormation.
- [**AWS Generative AI CDK Constructs**](https://awslabs.github.io/generative-ai-cdk-constructs/): A collection of CDK constructs for building generative AI applications on AWS.
- [**Powertools for AWS Lambda**](https://docs.powertools.aws.dev/lambda/python/latest/): A suite of utilities for AWS Lambda functions to ease the adoption of best practices such as tracing, structured logging, and custom metrics.


In the prior lab we built a quick agent using the smolagent library. This provided a quick way to get started with agents, but it is not a production ready solution. In this lab we will build a more robust agent using Amazon Bedrock and AWS CDK. This will allow us to deploy the agent in a production ready manner and take advantage of the AWS ecosystem. There is a bit more setup involved, but it will be worth it in the end. We will also use the AWS Generative AI CDK Constructs to simplify the process of building the agent. 

The architecture of the agent is shown in the diagram below. The agent will use the following components:
- **Bedrock Agent**: The agent that will be used to analyze financial news articles and provide insights on stock performance.
- **Action Groups**: A collection of actions that the agent can perform implemented via Lambda functions. In this case, we will have two action groups: one for analyzing financial news articles and one for providing insights on stock performance.

![Architecture Diagram](lab_assets/bedrock_agent_architecture.png)

We'll need to do the following steps to migrate our smolagent to Amazon Bedrock Agents:
- ✅ Convert the smolagent tool code to Lambda functions
- ❌ Generate OpenAPI spec for the Lambda functions to inform the agent about the tools and their parameters
- ❌ Create a Lambda environment that has all of the dependencies needed to run the tools
- ❌ Deploy the infrastructure including the Lambda functions, Bedrock agent, and any other resources needed such as IAM roles, S3 buckets, etc.
- ❌ Test the agent to ensure it is working as expected

### Setup and explore the development environment


Our first order of business is to create our development environment. 

**We'll run the following command to build the development environment** `make setup-environment`

This command will use the [uv](https://github.com/astral-sh/uv) package manager to create a virtual environment in the `.venv` directory. It will also install the [AWS CDK](https://docs.aws.amazon.com/cdk/v2/guide/getting_started.html) and a Docker client which we will use to build our Lambda functions.



In [ ]:
%%bash
make setup-environment


### Project Structure
Our project structure looks like this:
```
├── 1_bedrock_agent_lab.ipynb         # Notebook for the lab
├── action_handlers                   # Directory for the Lambda functions that implement the actions the agent can perform
│   └── stock_data                    # Directory for the Lambda function that contains tools to retrieve and analyze stock data    
│       ├── Dockerfile                # Dockerfile for the Lambda function
│       ├── lambda_code               # Directory for the Lambda function code
│       │   ├── index.py
│       │   ├── __init__.py
│       ├── requirements.txt          # Requirements for the Lambda function
│       └── test.py                   # Test file for the Lambda function
├── cdk_app.py                        # CDK entry point for building the infrastructure
├── cdk.context.json                  # CDK context file used to cache context values
├── cdk.json                          # CDK configuration file used to configure the CDK app
├── infra                             # Directory for the CDK stack that contains the infrastructure definition for the agent
│   └── agent_stack.py
├── Makefile                          # Makefile for the project to simplify the setup and deployment process in lieu of running long commands
├── pyproject.toml                    # Project configuration file for the python environment   
├── scripts                           # Directory for the scripts used to set up the environment
│   ├── install_cdk.sh
│   ├── install_docker.sh
└── uv.lock                           # Lock file for the uv package manager, helps to ensure the environment is consistent across different machines
```

This structure enables us to progressively build the agent functionality and deploy it using the AWS CDK. For example we can introduce new action groups by creating new directories under `action_handlers` and adding the new action groups to the CDK stack. Similarly we can add additional infrastructure resource such as S3 buckets, DynamoDB tables, OpenSearch Collections, etc. to the `infra` directory and add them to the CDK stack. 

### Explore the Lambda Code
Open the [action_handlers/stock_data/lambda_code/index.py](action_handlers/stock_data/lambda_code/index.py) file. This file contains the code for the Lambda function that will be used to analyze financial news articles and provide insights on stock performance.

There are a few things to note about the code:
The code uses the [AWS Lambda Powertools](https://docs.powertools.aws.dev/lambda/python/latest/) library to simplify the logging and tracing of the Lambda function. This is a best practice for AWS Lambda functions and will help us to debug the function if there are any issues.

Few other things to note:
- First we instantiate an app via `app = BedrockAgentResolver()`. The [agent resolver](https://docs.powertools.aws.dev/lambda/python/latest/core/event_handler/bedrock_agents/#your-first-agent) is provided via the AWS Lambda Powertools library and is used to resolve the tool or function that the agent is requesting to invoke. The resolver allows us to implement a very simple handler function that will be used to route the agent's request to the appropriate tool or function.
    ```python
    def lambda_handler(event: dict, context: LambdaContext):
        return app.resolve(event, context)

- Each function is wrapped with an `@app.get()` decorator. This decorator does not change the functionality but rather informs the agent on the available paths and the parameters that are available for each path. The agent will use this information to determine which function to call and what parameters to pass to it.
    ```python
    @app.get(
        "/get_ticker_data",
        description="Downloads historical stock data from Yahoo Finance and returns it as a dictionary",
        operation_id="getTickerData",
    )
    def get_ticker_data(...):
    ```
- Each parameter in the function is annotated with a type hint and additional `Query` meta-data. This metadata is used to inform the agent about the parameters that are available for each function. The agent will use this information to determine which parameters to pass to the function. In the example below, tickers is a `str` type that takes in a stock symbol such as `AAPL` or `MSFT`.
    ```python
     tickers: Annotated[
        str,
        Query(
            description="A stock ticker symbol",
            examples=["AAPL", "MSFT"],
            alias="ticker",
        ),
    ],
    ```
- The function return is also annotated with a type hint and additional metadata pertaining to the `Body` of the return. In the example below the function returns a dictionary where the keys are ticker symbols and the values are lists of historical data records. The agent will use this information to determine the return type of the function.

    ```python
    def get_ticker_data(...) -> Annotated[
        dict,
        Body(
            description="A dictionary where keys are ticker symbols and values are lists of historical data records"
        ),
    ]:
    ```

- All of the annotations are used to automatically generate an [OpenAPI spec](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-api-schema.html) for the Lambda function. This spec is used to inform the agent about the available paths and the parameters that are available for each path. To generate the spec, we simply run the lambda code as a python script. The main function in the `index.py` file is used to generate the spec and save it to a file. We can the use the file when deploying the agent infrastructure. 
    ```python
   if __name__ == "__main__":
    from pathlib import Path

    (Path(__file__).parent.parent / "api_schema.json").write_text(
        json.dumps(
            json.loads(app.get_openapi_json_schema(title="Stock Data API")), indent=2
        )
    )
    ```

The approach above enables us to keep the code DRY and allows us to easily add new functions and parameters to the Lambda function without having to manually update the OpenAPI spec. The agent will automatically pick up the new functions and parameters and use them in the requests.

Let's generate the spec by running the code below.

In [ ]:
%%bash
uv run python action_handlers/stock_data/lambda_code/index.py

Once you've run the cell above, you should see a new file called `api_schema.json` in the `action_handlers/stock_data/` directory. Click [here](action_handlers/stock_data/api_schema.json) to view the file. As you can see, the file contains the OpenAPI spec for the Lambda function. The spec contains the available paths, parameters, and return types for each function. Using the Annotations provided by the AWS Lambda Powertools library is far easier than having to generate the spec by hand especially for function involving more complex input types. As an added benefit we can use pydantic library to perform input and output [validation](https://docs.powertools.aws.dev/lambda/python/latest/core/event_handler/bedrock_agents/#validating-input-and-output) rather than having to write our own validation code.

### Locally testing the Lambda function
The Lambda function can be tested locally using the `pytest` framework. The test file is located in the [action_handlers/stock_data/tests.py](action_handlers/stock_data/tests.py) file. We can test the individual tool functions as well as the Lambda handler itself. Run the cell below to run the tests. 

**Note: Test may fail due to API limits**

In [ ]:
%%bash 
uv run pytest action_handlers/stock_data/tests.py 

### At this point we have a working Lambda function that can be used to analyze financial news articles and provide insights on stock performance. The next step is to deploy the Lambda function and the agent infrastructure using the AWS CDK.

### Deploy the infrastructure
The infrastructure for the agent is defined in the [infra/agent_stack.py](infra/agent_stack.py) file. To simplify the deployment, the stack uses [AWS Generative AI CDK Constructs](https://awslabs.github.io/generative-ai-cdk-constructs/) to create the agent and the action groups. These constructs reduce the amount of code needed to create the agent and the action groups by implicitly creating the necessary resources such as IAM roles, S3 buckets, and DynamoDB tables. 

The `agent_stack.py` is imported into the `app.py` file which serves as the entry point for the CDK app. This allows us to define additional stacks in the future if needed. 

To deploy the agent we need to run the following commands:
`cdk bootstrap` - This command will create the necessary resources for the CDK app. This includes creating an S3 bucket to store the CloudFormation templates and a DynamoDB table to store the state of the CDK app. The command will also create an IAM role that will be used to deploy the stack. It is only needed to be run once per AWS account and region.

`cdk synth` - This command will generate the CloudFormation template for the stack. The template will be saved in the `cdk.out` directory. This command can be run multiple times to update the stack.

`cdk deploy` - This command will deploy the stack to the AWS account. The command will create the necessary resources for the agent and the action groups. The command will also create an IAM role that will be used to invoke the Lambda functions. The command will prompt you to confirm the deployment before proceeding.

CDK will handle all of the heavy lifting for us including provisioning the infrastructure, creating the Lambda functions, building the custom Docker images, and creating the agent. Run the cell below to deploy the stack.

In [ ]:
%%bash
source ~/.bashrc
cd -
cdk bootstrap

In [ ]:
%%bash
source ~/.bashrc
cd -
cdk synth

In [ ]:
%%bash
source ~/.bashrc
cd -
cdk deploy --ci --require-approval never

### Invoke the agent
Once the stack is deployed, we can invoke the agent using the AWS APIs. The agent will be able to analyze financial news articles and provide insights on stock performance. The agent will use the action groups to perform the analysis and return the results.

The code cell below will lookup the agent_id and construct the AWS Console URL that you can click to open the agent in the AWS Console for testing and debugging.

In [ ]:
%pip install -Uq boto3

In [ ]:
import boto3
import json
from xray_utils import create_xray_segments_from_events
from rich import print as rprint
from rich.markdown import Markdown


# get the ARN of the deployed Bedrock agent
cfn_client = boto3.client("cloudformation")
stack_name = "BedrockAgentsStack"
response = cfn_client.describe_stacks(StackName=stack_name)
stack_outputs = response["Stacks"][0]["Outputs"]
for output in stack_outputs:
    if output["OutputKey"] == "BedrockAgentArn":
        bedrock_agent_arn = output["OutputValue"]



bedrock_agent_id = bedrock_agent_arn.split("/")[-1]
deployment_region = bedrock_agent_arn.split(":")[3]
bedrock_agent_alias = "TSTALIASID" # alias for the working version of the agent

bedrock_agent_url = f"https://{deployment_region}.console.aws.amazon.com/bedrock/home?region={deployment_region}#/agents/{bedrock_agent_id}"

bedrock_agent_runtime = boto3.client("bedrock-agent-runtime")
xray_client = boto3.client('xray')

rprint(Markdown(f"**You can click [here]({bedrock_agent_url}) to view and test the Bedrock agent in the AWS console.**"))

Next let's invoke the agent programmatically using the `invoke_agent` API. We provide a unique `session_id` to the API which will be used to store the conversation history and context. The agent will use the session_id to keep track of the conversation and provide context for the analysis. Additionally, we `enableTrace` in the API call which will emit detailed trace events of all the actions performed by the agent. This will help us to debug the agent and understand how it is performing the analysis. The trace events will be logged using [AWS X-Ray](https://aws.amazon.com/xray/) and can be viewed in the CloudWatch Console.

In [ ]:
import uuid

session_id = str(uuid.uuid4())

resp = bedrock_agent_runtime.invoke_agent(
    agentId=bedrock_agent_id,
    agentAliasId=bedrock_agent_alias,
    enableTrace=True,
    sessionId=session_id,
    inputText="How has Amazon's stock performed since the start of the year?",
)

# we capture all of the trace events emitted by the agent into a python list
event_stream = resp['completion']
trace_events = []
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            rprint(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            # rprint(event['trace'])
            trace_events.append(event['trace'])
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

In [ ]:
# example of a trace event
trace_events[0]

In [ ]:
# X-ray segments have to be in a specific format
# The trace events emitted by the agent have to be converted to X-ray segments
xray_segments = create_xray_segments_from_events(trace_events)
trace_id = json.loads(xray_segments[0])["trace_id"]

xray_resp = xray_client.put_trace_segments(
    TraceSegmentDocuments=xray_segments
)


trace_url = f"https://{deployment_region}.console.aws.amazon.com/xray/home?region={deployment_region}#/traces/{trace_id}"
rprint(Markdown(f"**You can click [here]({trace_url}) to view the trace events in the AWS console.**"))

In [ ]:
%%bash
source ~/.bashrc
cd -
cdk destroy -f